
# Projeto Prático - Pipeline Python para Dados via APIs

## Objetivo do projeto

#### Construir um pipeline de extração → tratamento → disponibilização de dados usando Python (ou PySpark/Databricks, conforme preferência). Cada participante deve extrair dados de pelo menos duas APIs públicas distintas, realizar transformações e entregá-los em formato consumível via GIT (CSV / Parquet / tabela em Snowflake/S3 / REST endpoint, Dashboard e entre outros de sua preferência) disponibilizando os dados da melhor maneira possível, aplicando práticas de Engenharia de Dados (orquestração, particionamento, testes de qualidade, documentação).

##### Desenvolvido por: Douglas Santos Azambuja

In [0]:
# importar Bibliotecas 

import requests
import pandas as pd
import pyspark as spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import pearsonr
import statsmodels.api as sm

In [0]:
# URL da API
url = "https://disease.sh/v3/covid-19/countries"

# Faz a chamada GET
response = requests.get(url, headers={"accept": "application/json"})

# Verifica se deu certo
if response.status_code == 200:
    data = response.json()
    
    # Normaliza JSON aninhado para DataFrame
    df_covid = pd.json_normalize(data)
    
    # Exibe as primeiras linhas
    display(df_covid)
else:
    print("Erro:", response.status_code, response.text)

In [0]:
# Verificando o tipo de dados

df_covid.info()

In [0]:
# Verificando a quantidade de campos nulos

df_covid.isnull().sum()

In [0]:
## Importando a segunda API com dos dados relacionados a quantidade de vacinas de covid aplicadas

# URL da API
url = "https://disease.sh/v3/covid-19/vaccine/coverage/countries?lastdays=1"

# Faz a chamada GET
response = requests.get(url, headers={"accept": "application/json"})

# Verifica se deu certo
if response.status_code == 200:
    data = response.json()
    
    # Normaliza JSON aninhado para DataFrame
    df_vacina = pd.json_normalize(data)
    
    # Exibe as primeiras linhas
    display(df_vacina)
else:
    print("Erro:", response.status_code, response.text)

In [0]:
# Verificando se tem campos nulos

df_vacina.isnull().sum()

In [0]:
# Renomeando o nome do campo para vaccina

df_vacina = df_vacina.rename(columns={"timeline.7/21/25": "vaccina"})

display(df_vacina)

In [0]:
# Fazendo um left join pela coluna "country"

df_consolidado = pd.merge(df_covid, df_vacina, on='country', how='left')

# Quando fiz o merge o tipo da coluna foi alterado para float
df_consolidado["vaccina"] = df_consolidado["vaccina"].astype("Int64")

display(df_consolidado)

In [0]:
# Selecionando apenas as colunas que acredito ser relevantes para o estudo

df_consolidado1 = df_consolidado[["continent","country","population" ,"cases", "deaths","vaccina","recovered"]]
display(df_consolidado1)